# Construcción Curva OIS

In [5]:
from IPython.display import Image

En el notebook anterior vimos que el valor de un derivado perfectamente colateralizado se obtiene descontando su payoff con un factor de descuento que proviene de la curva asociada a la tasa de remuneración del colateral.

Dicha tasa de remuneración es, en la gran mayoría de los casos, una tasa overnight. En el caso USD, para los CSA se utiliza *Effective Fed Funds* mientras que las contrapartes centrales están migrando a *SOFR*.

A continuación se verá cómo construir la curva asociada a *SOFR* a partir de cotizaciones de swaps de *SOFR* versus tasa fija. El procedimiento se conoce como **bootstrapping** por el hecho que la construcción de la curva se hace de forma incremental, partiendo de una solución parcial de fácil obtención.

## Bootstrapping para OIS

Tenemos que (notebook 4):

$$
\begin{equation}
\Pi\left(t,X\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\Pi\left(T,X\right)\right]
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)=\exp\left[-\int_t^Tr_c\left(u\right)du\right]
\end{equation}
$$

$$
\begin{equation}
\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\right]=P_c\left(t,T\right)
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)={B_c\left(t,T\right)}^{-1}
\end{equation}
$$

Consideremos el valor de la pata fija de un OIS. Para fijar las ideas, supongamos una pata a 2Y con cupones anuales. Suponemos, adicionalmente que el valor del nocional es 1 y que éste se paga al vencimiento. El valor de la pata fija es:

$$
\begin{equation}
\Pi^{fija}\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\cdot r\cdot\frac{T_1-T_0}{360}+D_c\left(t,T_2\right)\left(1+r \cdot \frac{T_2-T_1}{360}\right)\right]
\end{equation}
$$

$$
\begin{equation}
\Pi^{fija}\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\right]\cdot r\cdot\frac{T_1-T_0}{360}+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_2\right)\right]\left(1+r \cdot \frac{T_2-T_1}{360}\right)
\end{equation}
$$

$$
\begin{equation}
\Pi^{fija}\left(t\right)=P_c\left(t,T_1\right)\cdot r\cdot\frac{T_1-T_0}{360}+P_c\left(t,T_2\right)\left(1+r \cdot \frac{T_2-T_1}{360}\right)
\end{equation}
$$

Vemos que la expresión (7) corresponde a la suma de:

- primer cupón de intereses traído a valor presente con el factor de descuento entre $t$ y $T_1$
- segundo cupón de intereses y nocional traído a valor presente con el factor de descuento entre $t$ y $T_2$

Donde $T_0$ representa la fecha en que el primer cupón del swap comienza a devengar intereses (usualmente 2 días hábiles después de la fecha de celebración del contrato).

Por otro lado, el valor de la pata flotante es:

$$
\begin{equation}
\Pi^{flot}\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right) \cdot \left(\frac{B_c\left(t,T_1\right)}{B_c\left(t,T_0\right)}-1\right)\right]+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_2\right) \cdot \frac{B_c\left(t,T_2\right)}{B_c\left(t,T_1\right)}\right]
\end{equation}
$$

Aplicando (4) se obtiene:

$$
\begin{equation}
\Pi^{flot}\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_0\right)-D_c\left(t,T_1\right)\right]+\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_1\right)\right]
\end{equation}
$$

Utilizando el hecho que el valor esperado es un operador lineal se llega a:

$$
\begin{equation}
\Pi^{flot}\left(t\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_0\right)\right]=P_c\left(t,T_0\right)
\end{equation}
$$

Si se supone que $t=T_0$ tenemos que:

$$
\begin{equation}
P_c\left(t,T_0\right)=P_c\left(T_0,T_0\right)=1
\end{equation}
$$

Y por lo tanto, utilizando (7) y (10) se deduce que:

$$
\begin{equation}
\Pi^{fija}\left(T_0\right)=P_c\left(t_0,T_1\right)\cdot r\cdot\frac{T_1-T_0}{360}+P_c\left(t_0,T_2\right)\left(1+r \cdot \frac{T_2-T_1}{360}\right)=1
\end{equation}
$$

O sea, el valor presente de los flujos de la pata fija, considerando que el nocional se paga al vencimiento y utilizando los factores de descuento de la curva OIS es 1 (más generalmente es igual al nocional si se considera un nocional distinto de 1).

La fórmula (12) se puede extender fácilmente a los OIS de cualquier vencimiento y la podemos escribir, en forma general, de la siguiente manera:

$$
\begin{equation}
\Pi^{fija,tenor}\left(T_0\right)=\sum\limits_{j=1}^{m-1}P_c\left(T_0,T_j\right)\cdot r\cdot\frac{T_j-T_{j-1}}{360}+P_c\left(T_m,T_{m-1}\right)\cdot\left(1+r\cdot\frac{T_m-T_{m-1}}{360}\right)=1
\end{equation}
$$

Donde $tenor$ representa el plazo del swap (6M, 1Y, 5Y, ...) y $m$ es el número de cupones del swap. Por ejemplo, en un swap con $tenor=5Y$ y periodicidad anual, $m=5$. Para el mismo $tenor$, si la periodicidad es semestral, entonces $m=10$.

La ecuación (13), por si sola, no sería muy útil si no existiera toda una familia de OIS a distintos plazos para los cuales puedo observar su tasa fija en un momento dado del tiempo. Por ejemplo, para los OIS de *SOFR*, se dispone del siguiente panel de precios durante las horas en que estos productos se transan (hay incluso más plazos, pero no se ven todos en esta imagen).

In [6]:
Image(url="img/20201015_sofr_swaps.gif", width=900, height=720)

Los primeros 13 swaps son cero cupón, y por lo tanto, las ecuaciones correspondientes son muy sencillas:

$$
\begin{equation}
\Pi^{fija,tenor}\left(T_0\right)=P_c\left(T_0,T_{tenor}\right)\cdot\left(1+r\cdot\frac{T_{tenor}-T_0}{360}\right)=1
\end{equation}
$$

donde $tenor\in\ \left\{ 1W, 2W, 3W, 1M, 2M, 3M, 4M, 5M, 6M, 9M, 10M, 11M,1Y \right\}$.

Al swap cuyo $tenor=18M$ le corresponde la siguiente ecuación:

$$
\begin{equation}
\Pi^{fija,18M}\left(T_0\right)=P_c\left(T_0,T_1\right)\cdot r\cdot\frac{T_1-T_{0}}{360}+P_c\left(T_0,T_{2}\right)\cdot\left(1+r\cdot\frac{T_2-T_1}{360}\right)=1
\end{equation}
$$

En (15), gracias a las ecuaciones (14), el único término desconocido es $P_c\left(T_0,T_2\right)$ y por lo tanto, se puede resolver. Esto mismo se repite para los swaps cuyo $tenor \in \left\{ 2Y,3Y,4Y,5Y \right\}$.

Al swap cuyo $tenor=10Y$ le corresponde la siguiente ecuación:

$$
\begin{equation}
\Pi^{fija,10Y}\left(T_0\right)=\sum\limits_{j=1}^{9}P_c\left(T_0,T_j\right)\cdot r\cdot\frac{T_j-T_{j-1}}{360}+P_c\left(T_0,T_{10}\right)\cdot\left(1+r\cdot\frac{T_{10}-T_9}{360}\right)=1
\end{equation}
$$

En (16), los términos $P_c\left(T_0,T_j\right)$ con $j\in \left\{6,7,8,9,10\right\}$y son desconocidos, lo que nos fuerza a imponer alguna condición adicional que reduzca el número de incógnitas a 1. La condición que se impone es la siguiente:

$$
\begin{equation}
P_c\left(T_0,T_j\right)=g\left(T_j, P_c\left(T_0,T_5\right),  P_c\left(T_0,T_{10}\right)\right)
\end{equation}
$$

Donde $j\in\left\{6,7,8,9\right\}$ y $g$ es algún tipo de interpolación. Un razonamiento similar debe aplicarse al swap cuyo $tenor=20Y$.

### Ejercicio

Demuestre que, para valorizar, el valor esperado en $t$ de un flujo entre $T_1$ y $T_2$ de la pata flotante se puede calcular como:

$$
\begin{equation}
nominal\cdot r_{T_1,T_2}\cdot\frac{T_2-T_1}{360}
\end{equation}
$$

donde

$$
\begin{equation}
r_{T_1,T_2}=\left( \frac{P_c\left(t, T_1\right)}{P_c\left(t, T_2\right)}-1\right)\cdot\frac{360}{T_2-T_1}
\end{equation}
$$

Por una parte tenemos que (considerando $nominal=1$):

$$
\begin{equation}
\Pi^{flot}\left(t,T_1,T_2\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T_2\right)\left(\frac{B_c\left(t,T_2\right)}{B_c\left(t,T_1\right)}-1\right)\right]=P_c\left(t,T_1\right)-P_c\left(t,T_2\right)
\end{equation}
$$

Por otra parte también sabemos que (siempre con $nominal=1$):

$$
\begin{equation}
\Pi^{flot}\left(t,T_1,T_2\right)=P_c\left(t,T_2\right)\cdot\mathbb{E}_t^{Q_f^{T_2}}\left[ON\left(T_1,T_2\right)\cdot\frac{T_2-T_1}{360}\right]
\end{equation}
$$

Donde $ON\left(T_1,T_2\right)$ es la tasa equivalente al producto de los factores de capitalización de la tasa overnight asociada al OIS entre $T_1$ y $T_2$.

Juntando (20) y (21) llegamos a:

$$
\begin{equation}
\frac{T_2-T_1}{360}\cdot\mathbb{E}_t^{Q_f^{T_2}}\left[ON\left(T_1,T_2\right)\right]=\frac{P_c\left(t,T_1\right)-P_c\left(t,T_2\right)}{P_c\left(t,T_2\right)}
\end{equation}
$$

Por lo tanto, si denotamos $\mathbb{E}_t^{Q_f^{T_2}}\left[ON\left(T_1,T_2\right)\right]=r_{T_1,T_2}$ (22) implica (18) utilizando $nominal=1$. Es evidente que el resultado se generaliza para cualquier $nominal>0$.

## Implementación

En lo que sigue, se implementa el **bootstrapping** para swaps de *SOFR* versus tasa fija.

### Librerías

In [7]:
from finrisk import QC_Financial_3 as Qcf
from scipy.optimize import root_scalar
import modules.auxiliary as aux
import pandas as pd
import math

### Variables Globales

Se da de alta un objeto de tipo `Qcf.BusinessCalendar`y se agregan los feriados relevantes para la implementación.

In [8]:
bus_cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
for agno in range(2020, 2071):
    f = Qcf.QCDate(12, 10, agno)
    if f.week_day() == Qcf.WeekDay.SAT:
        bus_cal.add_holiday(Qcf.QCDate(14, 10, agno))
        # pass
    elif f.week_day() == Qcf.WeekDay.SUN:
        bus_cal.add_holiday(Qcf.QCDate(13, 10, agno))
    elif f.week_day() == Qcf.WeekDay.MON:
        bus_cal.add_holiday(Qcf.QCDate(12, 10, agno))
    elif f.week_day() == Qcf.WeekDay.TUE:
        bus_cal.add_holiday(Qcf.QCDate(11, 10, agno))
    elif f.week_day() == Qcf.WeekDay.WED:
        bus_cal.add_holiday(Qcf.QCDate(10, 10, agno))
    elif f.week_day() == Qcf.WeekDay.THU:
        bus_cal.add_holiday(Qcf.QCDate(9, 10, agno))
    else:
        bus_cal.add_holiday(Qcf.QCDate(8,10, agno))
bus_cal.add_holiday(Qcf.QCDate(15, 2, 2021))

### Funciones

In [9]:
def make_fixed_leg(start_date: str, tenor: str, rate_value: float) -> Qcf.Leg:
    """
    Construye un pata fija con algunos parámetros prefijados:
    
    - recibo o pago: R
    - nocional: 1,000,000
    - moneda: USD
    - periodicidad: 1Y
    - business adjustment rule: MODFOLLOW
    - stub period: SHORTBACK
    
    params:
    
    - start_date: fecha inicial de la pata en formato ISO
    - tenor: plazo estructurado de la pata (1Y, 2Y, 18M, ...)
    - rate_value: valor de la tasa fija
    
    return:
    
    - objeto `Qcf.Leg` con cashflows de tipo `FixedRateCashflow`
    """
    # Recibo o pago los flujos de esta pata
    rp = Qcf.RecPay.RECEIVE

    # Periodicidad de pago
    periodicidad = Qcf.Tenor('1Y')

    # Tipo de período irregular (si lo hay)
    periodo_irregular = Qcf.StubPeriod.SHORTFRONT

    # Regla para ajustes de días feriados
    bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW

    # Número de días después de la fecha final de devengo en que se paga el flujo
    lag_pago = 0

    # Nocional del contrato
    nocional = 10000000.0

    # Considera amortización
    amort_es_flujo = True

    # Moneda
    moneda = Qcf.QCUSD()

    # Es un bono de RF
    es_bono = False

    # Fecha de inicio de devengo del primer cupón
    fecha_inicial = Qcf.build_qcdate_from_string(start_date)

    # Fecha final de devengo, antes de ajustes, del último cupón
    qc_tenor = Qcf.Tenor(tenor)
    meses = qc_tenor.get_years()*12 + qc_tenor.get_months()
    if meses > 0:
        fecha_final = fecha_inicial.add_months(meses)
    else:
        dias = qc_tenor.get_days()
        fecha_final = fecha_inicial.add_days(dias)

    valor_tasa = rate_value
    tasa_cupon = Qcf.QCInterestRate(
        valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    # Se da de alta el objeto y se retorna
    return Qcf.LegFactory.build_bullet_fixed_rate_leg(
        rp,
        fecha_inicial,
        fecha_final,
        bus_adj_rule,
        periodicidad,
        periodo_irregular,
        bus_cal,
        lag_pago,
        nocional,
        amort_es_flujo,
        tasa_cupon,
        moneda,
        es_bono)

In [10]:
def error_with_rate(rate: float, *args) -> float:
    """
    Calcula el error entre el valor presente de una pata fija calculado con una curva cero y el nocional
    de la pata fija. Esta función está diseñada para servir de apoyo al proceso de bootstrapping de una curva
    cero cupón de OIS.
    
    params:
    
    - rate: valor a utilizar para la última tasa de la curva
    - args: iterable con plazos de la curva, tasas de la curva, objeto Qcf.Leg a valorizar y fecha de
    valorización.
    
    return:
    
    - float con el monto del error.
    """
    # Para usar la librería, los plazos y tasas deben estar en estos formatos.
    # Recordar que la librería está escrita en C++. 
    plazos = Qcf.long_vec()
    for p in args[0]:
        plazos.append(p)
    
    tasas = Qcf.double_vec()
    for t in args[1]:
        tasas.append(t)
    
    pata = args[2]
    
    fecha_val = args[3]
    
    # Se construye una curva cero Qcf
    curva = Qcf.QCCurve(plazos, tasas)
    cuantos = curva.get_length()
    par = curva.get_values_at(cuantos - 1)
    curva.set_pair(par.tenor, rate) # La variable rate se transforma en la última tasa de la curva
    curva = Qcf.QCLinearInterpolator(curva)
    curva = Qcf.ZeroCouponCurve(curva, Qcf.QCInterestRate(
        0.0, Qcf.QCAct365(), Qcf.QCContinousWf()))
    
    # Se da de alta el objeto que calcula valores presentes
    vp = Qcf.PresentValue()
    
    # Se retorna la diferencia entre el vp calculado con la curva y el nominal
    return vp.pv(fecha_val, pata, curva) - pata.get_cashflow_at(0).get_nominal()

### Formatos `DataFrame`

In [11]:
frmt = {'rate_value': '{:.8%}', 'valor_tasa': '{:.8%}',
        'nominal': '{:,.2f}', 'amortizacion': '{:,.2f}',
        'interes': '{:,.2f}', 'flujo': '{:,.2f}'
       }

### Obtiene Data

In [12]:
swaps = pd.read_excel('data/20201012_sofr_zero.xlsx', sheet_name='upload')
swaps['start_date'] = swaps['start_date'].astype(str).str[:10]

In [13]:
swaps.style.format(frmt)

,start_date,tenor,rate_value,rate_type,periodicity,stub_period
0,2020-10-14,1D,0.08000000%,LINACT360,1Y,SHORTFRONT
1,2020-10-14,7D,0.08290000%,LINACT360,1Y,SHORTFRONT
2,2020-10-14,14D,0.07690000%,LINACT360,1Y,SHORTFRONT
3,2020-10-14,21D,0.07630005%,LINACT360,1Y,SHORTFRONT
4,2020-10-14,1M,0.07700000%,LINACT360,1Y,SHORTFRONT
5,2020-10-14,2M,0.07700000%,LINACT360,1Y,SHORTFRONT
6,2020-10-14,3M,0.08000000%,LINACT360,1Y,SHORTFRONT
7,2020-10-14,4M,0.07700000%,LINACT360,1Y,SHORTFRONT
8,2020-10-14,5M,0.07500000%,LINACT360,1Y,SHORTFRONT
9,2020-10-14,6M,0.07400000%,LINACT360,1Y,SHORTFRONT


### Construye Objetos `Qcf`

In [14]:
qc_swaps = []
for s in swaps.itertuples():
    leg = make_fixed_leg(s.start_date, s.tenor, s.rate_value)
    cuantos_flujos = leg.size()
    last_pmt_date = leg.get_cashflow_at(cuantos_flujos - 1).get_settlement_date()
    start_date = Qcf.build_qcdate_from_string(s.start_date)
    plazo = start_date.day_diff(last_pmt_date)
    qc_swaps.append((leg, plazo))

In [15]:
aux.show_leg(qc_swaps[16][0], 'FixedRateCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-10-14,2021-04-14,2021-04-14,"10,000,000.00",0.00,"3,200.17",True,"3,200.17",USD,0.06330000%,LinAct360
1,2021-04-14,2022-04-14,2022-04-14,"10,000,000.00","10,000,000.00","6,417.92",True,"10,006,417.92",USD,0.06330000%,LinAct360


### Construye Curva Cero Cupón

Demuestra función objetivo.

In [16]:
plazos = []
tasas = []
 
plazos.append(1) # Los plazos se calculan desde start_date (2020-10-14)
tasa = math.log(1 + .08 / 100 * 1 / 360)* 365.0 / 1
# 1 + .08% * 1/360 = exp(r * 1/365) -> log(1 + .08% * 1/360) * 365/1
tasas.append(tasa)

plazos.append(int(7))
tasas.append(.0)

# Puedo valorizar cualquiera de los swaps ya dado de alta.
error_with_rate(.0008, plazos, tasas, qc_swaps[1][0], Qcf.QCDate(14, 10, 2020))

7.768488567322493

In [17]:
plazos = []
tasas = []
 
plazos.append(1) # Los plazos se calculan desde start_date (2020-10-14)
tasa = math.log(1 + .08 / 100 * 1 / 360)* 365.0 / 1
# 1 + .08% * 1/360 = exp(r * 1/365) -> log(1 + .08% * 1/360) * 365/1
tasas.append(tasa)

for s in qc_swaps[1:]:
    plazos.append(s[1])
    tasas.append(0.0)
    
    x = root_scalar(
        error_with_rate,
        method='bisect',
        bracket=[0.0, .02],
        x0=.0008,
        args=(plazos, tasas, s[0], Qcf.QCDate(14, 10, 2020)),
        xtol=.00000000000000001
    )
    
    tasas[-1] = x.root

In [18]:
for_df = []
for p, t in zip(plazos, tasas):
    for_df.append((p,t))
df_curva = pd.DataFrame(for_df, columns=['plazo', 'tasa'])
df_curva['df'] = df_curva.apply(lambda row: math.exp(-row['plazo']*row['tasa']/365), axis=1)
df_curva.style.format({'tasa':'{:.8%}', 'df':'{:.8%}'})

,plazo,tasa,df
0,1,0.08111102%,99.99977778%
1,7,0.08405071%,99.99838808%
2,14,0.07796689%,99.99700953%
3,21,0.07735805%,99.99554936%
4,33,0.07806669%,99.99294216%
5,61,0.07806435%,99.98695448%
6,92,0.08110282%,99.97955973%
7,125,0.07805901%,99.97327104%
8,152,0.07602963%,99.96834336%
9,182,0.07501375%,99.96260288%


Tasa fwd entre t1 y t2 días.

In [19]:
from scipy.interpolate import interp1d

In [20]:
crv = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear')

In [28]:
t1 = 98
t2 = 456
r1 = crv(t1)
r2 = crv(t2)
df1 = math.exp(-r1 * t1 / 365)
df2 = math.exp(-r2 * t2 / 365)
fwd12 = (df1 / df2 - 1.0) * 360.0 / (t2 - t1)
print(f'{fwd12:.8%}')

0.06269389%


Se exporta a Excel este resultado.

In [21]:
df_curva.to_excel('data/20201012_built_sofr_zero.xlsx', index=False)

### Comprobación

Con el resultado (raíces) se construye un objeto `Qcf.ZeroCouponCurve`.

In [22]:
qc_plazos = Qcf.long_vec()
qc_tasas = Qcf.double_vec()
for p, t in zip(plazos, tasas):
    qc_plazos.append(p)
    qc_tasas.append(t)
curva_final = Qcf.QCCurve(qc_plazos, qc_tasas)
curva_final = Qcf.QCLinearInterpolator(curva_final)

# Se construye la curva
curva_final = Qcf.ZeroCouponCurve(
    curva_final,
    Qcf.QCInterestRate(0.0, Qcf.QCAct365(), Qcf.QCContinousWf())
)

Se calcula el valor presente de cada una de las patas fijas de los swaps.

In [23]:
vp = Qcf.PresentValue()
for i, s in enumerate(qc_swaps):
    pv = vp.pv(Qcf.QCDate(14, 10, 2020), s[0], curva_final)
    print(f'{i}: {pv:,.8f}')

0: 10,000,000.00000000
1: 10,000,000.00000000
2: 10,000,000.00000000
3: 10,000,000.00000000
4: 10,000,000.00000000
5: 10,000,000.00000000
6: 10,000,000.00000000
7: 10,000,000.00000000
8: 10,000,000.00000000
9: 10,000,000.00000000
10: 10,000,000.00000000
11: 10,000,000.00000000
12: 10,000,000.00000000
13: 10,000,000.00000000
14: 10,000,000.00000000
15: 10,000,000.00000000
16: 10,000,000.00000000
17: 10,000,000.00000000
18: 10,000,000.00000000
19: 10,000,000.00000000
20: 10,000,000.00000000
21: 10,000,000.00000000
22: 10,000,000.00000000
23: 10,000,000.00000000
24: 10,000,000.00000000
25: 10,000,000.00000000
26: 10,000,000.00000000
27: 10,000,000.00000000
28: 10,000,000.00000000
29: 10,000,000.00000000
30: 10,000,000.00000000
31: 10,000,000.00000000
32: 10,000,000.00000000
